In [1]:
%%capture
!pip install -U langchain-google-genai langgraph langchain-core langchain-community python-dotenv

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("No se encuentra la variable de entorno GOOGLE_API_KEY")



In [3]:
from dotenv import load_dotenv
import os

load_dotenv()  # carga .env

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    print("API Key cargada correctamente")
else:
    print("No se encontró GOOGLE_API_KEY en el archivo .env")


API Key cargada correctamente


In [4]:
from dotenv import load_dotenv
import os
import google.generativeai as genai
from google.generativeai import list_models 

load_dotenv() 
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY").strip() 

if GOOGLE_API_KEY:
    print("API Key cargada correctamente")
    print(f"Longitud de la clave: {len(GOOGLE_API_KEY)}") # <--- ¡MIDE AQUÍ!
    
    try:
        genai.configure(api_key=GOOGLE_API_KEY) 
        models = list_models() 
        print("\nModelos disponibles:")
        for model in models:
            print(f"- {model.name} | Métodos soportados: {model.supported_generation_methods}")
            
    except Exception as e:
        print("\n❌ Aún falla la conexión. El error es probablemente la clave API o la red.")
        print(f"Detalle del error: {e}")
        
else:
    print("No se encontró GOOGLE_API_KEY en el archivo .env")

API Key cargada correctamente
Longitud de la clave: 39

Modelos disponibles:
- models/embedding-gecko-001 | Métodos soportados: ['embedText', 'countTextTokens']
- models/gemini-2.5-flash | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
- models/gemini-2.5-pro | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
- models/gemini-2.0-flash-exp | Métodos soportados: ['generateContent', 'countTokens', 'bidiGenerateContent']
- models/gemini-2.0-flash | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
- models/gemini-2.0-flash-001 | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
- models/gemini-2.0-flash-exp-image-generation | Métodos soportados: ['generateContent', 'countTokens', 'bidiGenerateContent']
- models/gemini-2.0-flash-lite-001 | Métodos soportados: ['generateContent', 'co

In [5]:
from typing import TypedDict, Annotated, Sequence
import operator
class AgentState(TypedDict):
    messages: Annotated[Sequence[dict], operator.add]
    current_step: str
    analysis_results: dict

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0.3,
    google_api_key=GOOGLE_API_KEY
) 

In [7]:
def analyze_input_gemini(state: AgentState, llm_instance):
      
    messages = state["messages"]
    

    gemini_messages = [
      
        {"role": "system", "content": "Eres un analista experto. Analiza la solicitud del usuario y determina qué tipo de ayuda necesita."},*messages
        
    ]

    response = llm_instance.invoke(gemini_messages)
  
    
    print(f" Análisis completado: {response.content[:100]}...") 
   

    return {
        
        "current_step": "analysis_complete",
        
        "analysis_results": {
            
            "intent": "technical_support",
           
            "confidence": 0.92,
            
            "details": response.content
           
        }
    }


def generate_response_gemini(state: AgentState, llm_instance):
        
    messages = state["messages"]
       
    analysis = state["analysis_results"]
    
    system_prompt = f"""
Eres un asistente técnico experto. El análisis indica:
- Intención: {analysis['intent']}
- Confianza: {analysis['confidence']:.2f}
- Detalles: {analysis['details'][:200]}

Proporciona una respuesta clara, técnica y útil. Sé conciso pero completo.
"""
    gemini_messages = [
        {"role": "system", "content": system_prompt},*messages
    ]

    response = llm_instance.invoke(gemini_messages)
    
    print(f"Respuesta generada: {response.content[:100]}...") 
   

    return {
        
        "current_step": "response_generated",
        
        "messages": [{"role": "assistant", "content": response.content}]
    }

In [16]:
def analyze_node(state):
    return analyze_input_gemini(state, llm)
def respond_node(state):
    return generate_response_gemini(state, llm)

In [17]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(AgentState)
workflow.add_node("analyze",analyze_node)
workflow.add_node("respond",respond_node)
workflow.add_edge(START,"analyze")
workflow.add_edge("analyze","respond")
workflow.add_edge("respond",END)



In [18]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [23]:
config = {"configurable": {"thread_id": "colab_test_1"}}

initial_state = {
   
    "messages": [
      
        {"role": "user", "content": "¿Cómo puedo optimizar el rendimiento de mi aplicación Flask?"}
        
    ],
    "current_step": "initial",
    
    "analysis_results": {}
    
}


print("Ejecutando agente con Gemini en LangGraph...")


result = app.invoke(initial_state, config=config)


Ejecutando agente con Gemini en LangGraph...
 Análisis completado: ¡Hola de nuevo!

Parece que me has vuelto a hacer la misma pregunta. Justo antes te proporcioné una ...
Respuesta generada: ¡Hola de nuevo!

Parece que me has vuelto a hacer la misma pregunta. Justo antes te proporcioné una ...


In [24]:
print("\n" + "="*50)

print("RESULTADO FINAL DEL AGENTE")

print("="*50)

print(f"Paso actual: {result['current_step']}")

print(f"Análisis: {result['analysis_results']}")

print("\n RESPUESTA DEL AGENTE:")

print(result["messages"][-1]["content"])



RESULTADO FINAL DEL AGENTE
Paso actual: response_generated
Análisis: {'intent': 'technical_support', 'confidence': 0.92, 'details': '¡Hola de nuevo!\n\nParece que me has vuelto a hacer la misma pregunta. Justo antes te proporcioné una respuesta muy completa y detallada sobre cómo optimizar tu aplicación Flask, cubriendo puntos clave como:\n\n*   **Optimización de Base de Datos** (índices, consultas eficientes, pooling).\n*   **Caching** (con `Flask-Caching`, Redis, Memcached).\n*   **Servidor de Producción y Concurrencia** (Gunicorn/uWSGI, Nginx/Apache).\n*   **Procesamiento Asíncrono** (Celery).\n*   **Optimización del Código Python/Flask** (perfilado, menos bloqueo).\n*   **Servir Archivos Estáticos** correctamente.\n*   **Monitoreo** (APM, Prometheus/Grafana).\n\nPuedes consultar la respuesta anterior para ver todos los detalles y las recomendaciones específicas.\n\n¿Hay alguna sección específica de esa respuesta que te gustaría que profundice, o tienes alguna pregunta más concreta